In [27]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [15]:
!wget http://deeplearning.net/data/mnist/mnist.pkl.gz -P drive/app

--2018-05-19 22:07:21--  http://deeplearning.net/data/mnist/mnist.pkl.gz
Resolving deeplearning.net (deeplearning.net)... 132.204.26.28
Connecting to deeplearning.net (deeplearning.net)|132.204.26.28|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16168813 (15M) [application/x-gzip]
Saving to: ‘drive/app/mnist.pkl.gz.1’

mnist.pkl.gz.1      100%[===================>]  15.42M  22.8MB/s    in 0.7s    

2018-05-19 22:07:22 (22.8 MB/s) - ‘drive/app/mnist.pkl.gz.1’ saved [16168813/16168813]



In [40]:
mnist = input_data.read_data_sets("drive/app/MNIST_data/", one_hot=True)

Extracting drive/app/MNIST_data/train-images-idx3-ubyte.gz
Extracting drive/app/MNIST_data/train-labels-idx1-ubyte.gz
Extracting drive/app/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting drive/app/MNIST_data/t10k-labels-idx1-ubyte.gz


In [0]:
# Training Parameters
learning_rate = 0.001
num_epochs = 15
batch_size = 128

# Data Parameters
num_input = 784  # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)

In [0]:
# TF Estimator input is a dict, in case of multiple inputs
X = tf.placeholder("float", [batch_size, num_input])
Y = tf.placeholder("float", [batch_size, num_classes])

# Reshape to match picture format [Height x Width x Channel]
# Tensor input become 4-D: [Batch Size, Height, Width, Channel]
x = tf.reshape(X, shape=[batch_size, 28, 28, 1])

# Convolution Layer with 32 filters and a kernel size of 5
conv1 = tf.layers.conv2d(x, 32, 5, activation=tf.nn.relu)
# Max Pooling (down-sampling) with strides of 2 and kernel size of 2
conv1 = tf.layers.max_pooling2d(conv1, 2, 2)

# Convolution Layer with 64 filters and a kernel size of 3
conv2 = tf.layers.conv2d(conv1, 64, 3, activation=tf.nn.relu)
# Max Pooling (down-sampling) with strides of 2 and kernel size of 2
conv2 = tf.layers.max_pooling2d(conv2, 2, 2)

# Flatten the data to a 1-D vector for the fully connected layer
fc1 = tf.contrib.layers.flatten(conv2)

# Fully connected layer (in tf contrib folder for now)
fc1 = tf.layers.dense(fc1, 1024)

# Output layer, class prediction
out = tf.layers.dense(fc1, num_classes)

# Define loss and optimizer
loss_op = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(logits=out, labels=Y)
)

# Optimization
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_op)

# Evaluate the accuracy of the model
pred = tf.nn.softmax(out)
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))


In [43]:
with tf.Session() as sess:

    sess.run(tf.global_variables_initializer())
    
    # Training cycle
    for epoch in range(num_epochs):

        avg_cost = 0.0
        
        total_batch = int(mnist.train.num_examples / batch_size)

        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)

            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, loss_op], feed_dict={X: batch_x, Y: batch_y})

            # Compute average loss
            avg_cost += c / total_batch

        # Display logs per epoch step
        if epoch % 1 == 0:
            print('Epoch: %04d, cost: %4.10f' % (epoch + 1, avg_cost))

    # Calculate accuracy
    avg_acc = 0.0
    
    total_test_batch = int(mnist.test.num_examples / batch_size)
    
    # Test cycle
    for i in range(total_test_batch):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        batch_x, batch_y = mnist.test.next_batch(batch_size)
        avg_acc += sess.run(accuracy, feed_dict={X: batch_x, Y: batch_y})
    
    print('Accuracy: %4.10f' % (avg_acc / total_test_batch))

Epoch: 0001, cost: 0.1520659345
Epoch: 0002, cost: 0.0460075843
Epoch: 0003, cost: 0.0327306254
Epoch: 0004, cost: 0.0259933803
Epoch: 0005, cost: 0.0219824929
Epoch: 0006, cost: 0.0175281693
Epoch: 0007, cost: 0.0176041371
Epoch: 0008, cost: 0.0143590284
Epoch: 0009, cost: 0.0125266345
Epoch: 0010, cost: 0.0137665005
Epoch: 0011, cost: 0.0112521511
Epoch: 0012, cost: 0.0110344613
Epoch: 0013, cost: 0.0084478811
Epoch: 0014, cost: 0.0095168456
Epoch: 0015, cost: 0.0107860034
Accuracy: 0.9905849359
